### Challenge 6:  Break repeating-key XOR

<div class="alert alert-block alert-info">
    
It is officially on, now.

This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.

There's a file [here](challenge-data/4.txt).

It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1.  Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
2.  Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between ```this is a test``` and ```wokka wokka!!!``` is 37. Make sure your code agrees before you proceed. 
3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
7. Solve each block as if it was single-character XOR. You already have code to do this.
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.

<div class="alert alert-block alert-warning">

### No, that's not a mistake.

 We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.


</div>
    
</div>

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
import base64
import cryptopals as cp

2.  Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
    
    
            this is a test
            
    and

            wokka wokka!!!
            

    is 37. Make sure your code agrees before you proceed. 


In [2]:
def hamming_distance(a, b):

    HD = 0
    for ii in range(len(a)):
        HD += bin(a[ii] ^ b[ii]).count('1')

    return(HD)

In [3]:
# Test Hamming Distance function:

a = b'this is a test'
b = b'wokka wokka!!!'

if hamming_distance(a,b) == 37:
    print('Test of hamming_distance function PASSED')
else:
    print('Test of hamming_distance function FAILED')


Test of hamming_distance function PASSED


In [4]:
# Read the challenge file in.
f = open('./challenge-data/6.txt', 'r')
encrypted_data = base64.b64decode(f.read())
f.close()

In [5]:
# 3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of 
#    bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
# 
# 4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed 
#    perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and 
#    average the distances.

keySizeScores = [0]*40
for KEYSIZE in range(1, 41):
    
    a = encrypted_data[0:KEYSIZE]
    b = encrypted_data[KEYSIZE:2*KEYSIZE+1]
    c = encrypted_data[KEYSIZE*2:3*KEYSIZE+1]
    d = encrypted_data[KEYSIZE*3:4*KEYSIZE+1]
    
    keySizeScores[KEYSIZE-1] = (cp.hamming_distance(a,b) + \
                                cp.hamming_distance(a,c) + \
                                cp.hamming_distance(a,d) + \
                                cp.hamming_distance(b,c) + \
                                cp.hamming_distance(b,d) + \
                                cp.hamming_distance(c,d))/ \
                                (6*KEYSIZE)
    
    KEYSIZE = int(cp.argmin(keySizeScores)+1)

In [6]:
# 5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
#
# 6. Now transpose the blocks: make a block that is the first byte of every block, and a block 
#    that is the second byte of every block, and so on.
#
# 7. Solve each block as if it was single-character XOR. You already have code to do this.
# 
# 8. For each block, the single-byte XOR key that produces the best looking histogram is the 
#    repeating-key XOR key byte for that block. Put them together and you have the key.

KEY = []
for key_idx in range(KEYSIZE):
    chunk = encrypted_data[key_idx::KEYSIZE]
    (chunk_key, _) = cp.break_single_char_XOR(chunk)
    KEY.append(chunk_key)

In [7]:
plaintext = cp.encrypt_repeating_key_XOR(encrypted_data, KEY)
print()
print(bytes(plaintext).decode())


I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)